In [1]:
import * as torch from "./index.ts";

const a = new torch.Tensor([1, 2, 3], [3, 1], true);
a._name = "a_tensor";
const b = new torch.Tensor([4, 5, 6], [3, 1], true);
b._name = "b_tensor";
const c = torch.add(a, b);
c._name = "c_tensor";
console.log("a:", a.toString());
console.log("b:", b.toString());
console.log("c:", c.toString());

a: Tensor(shape=[3,1])
b: Tensor(shape=[3,1])
c: Tensor(shape=[3,1])


In [2]:
const MN = new torch.Tensor(
  [
    2, 3,
    3, 7
  ]
, [2, 2])
MN.toString()

"Tensor(shape=[2,2])"

In [3]:
c.requires_grad

true

In [4]:
const d = torch.mul(a, c);
d._name = "d_tensor";
const e = torch.matmul(d, b.transpose());
e._name = "e_tensor";

"e_tensor"

In [5]:
d.backward(new Float32Array([0.001, 0.002, 0.003]));
// e.backward(new Float32Array([0.2, 0.8, 0.7]));

In [6]:
function logPrevs(tensor: torch.Tensor, indent: string = "") {
  if (tensor._prev) {
    for (let i = 0; i < tensor._prev.length; i++) {
      const prev = tensor._prev[i];
      console.log(`${indent}Prev Tensor: ${prev.toString()} (Name: ${prev?._name}), Grad: ${prev._grad ? prev._grad.toString() : "None"}`);
      logPrevs(prev, indent + "|  ");
    }
  }
}

function logTree(tensor: torch.Tensor) {
  console.log("Gradient computation graph:");
  console.log(`Tensor: ${tensor.toString()} (Name: ${tensor?._name})`);
  logPrevs(tensor, "|  ");
}

logTree(d);

Gradient computation graph:
Tensor: Tensor(shape=[3,1]) (Name: d_tensor)
|  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: 0.0060000005178153515,0.018000001087784767,0.035999998450279236
|  Prev Tensor: Tensor(shape=[3,1]) (Name: c_tensor), Grad: 0.0010000000474974513,0.004000000189989805,0.008999999612569809
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: 0.0060000005178153515,0.018000001087784767,0.035999998450279236
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: b_tensor), Grad: 0.0010000000474974513,0.004000000189989805,0.008999999612569809


In [7]:
logTree(e);

Gradient computation graph:
Tensor: Tensor(shape=[3,3]) (Name: e_tensor)
|  Prev Tensor: Tensor(shape=[3,1]) (Name: d_tensor), Grad: 0.0010000000474974513,0.0020000000949949026,0.003000000026077032
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: 0.0060000005178153515,0.018000001087784767,0.035999998450279236
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: c_tensor), Grad: 0.0010000000474974513,0.004000000189989805,0.008999999612569809
|  |  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: 0.0060000005178153515,0.018000001087784767,0.035999998450279236
|  |  |  Prev Tensor: Tensor(shape=[3,1]) (Name: b_tensor), Grad: 0.0010000000474974513,0.004000000189989805,0.008999999612569809
|  Prev Tensor: Tensor(shape=[1,3]) (Name: undefined), Grad: None


## Simple Neural Network

In [8]:
let x = new torch.Tensor([7, 8, 9,5],[1,4]);
x._name = "Input (x)"

let W_1 = torch.Tensor.randn([4,8], true);
W_1._name = "W_1"
let B_1 = torch.Tensor.randn([1,8], true);
B_1._name = "W_2"

console.log(x.shape, W_1.shape, B_1.shape)

[ 1, 4 ] [ 4, 8 ] [ 1, 8 ]


In [9]:
let h_1 = torch.add(torch.matmul(x, W_1), B_1);
h_1._name = "h_1"

let W_2 = torch.Tensor.randn([8,2], true);
W_2._name = "W_2"
let B_2 = torch.Tensor.randn([1,2], true);
B_2._name = "B_2"

console.log(h_1.shape, W_2.shape, B_2.shape)

[ 1, 8 ] [ 8, 2 ] [ 1, 2 ]


In [10]:
let h_2 = torch.add(torch.matmul(h_1, W_2), B_2);
h_2._name = "h_2"

"h_2"

In [11]:
h_2

Tensor {
  data: Float32Array(2) [ 56.78911590576172, 4.265871047973633 ],
  shape: [ 1, 2 ],
  requires_grad: true,
  _grad: undefined,
  _prev: [
    Tensor {
      data: Float32Array(2) [ 56.6873779296875, 5.548368453979492 ],
      shape: [ 1, 2 ],
      requires_grad: true,
      _grad: undefined,
      _prev: [
        Tensor {
          data: [Float32Array],
          shape: [Array],
          requires_grad: true,
          _grad: undefined,
          _prev: [Array],
          _name: "h_1",
          _backward: [Function (anonymous)]
        },
        Tensor {
          data: [Float32Array],
          shape: [Array],
          requires_grad: true,
          _grad: undefined,
          _prev: [],
          _name: "W_2",
          _backward: undefined
        }
      ],
      _name: undefined,
      _backward: [Function (anonymous)]
    },
    Tensor {
      data: Float32Array(2) [ 0.1017376258969307, -1.2824971675872803 ],
      shape: [ 1, 2 ],
      requires_grad: true,
      

In [12]:
logTree(h_2);

Gradient computation graph:
Tensor: Tensor(shape=[1,2]) (Name: h_2)
|  Prev Tensor: Tensor(shape=[1,2]) (Name: undefined), Grad: None
|  |  Prev Tensor: Tensor(shape=[1,8]) (Name: h_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: undefined), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[1,4]) (Name: Input (x)), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[4,8]) (Name: W_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: W_2), Grad: None
|  |  Prev Tensor: Tensor(shape=[8,2]) (Name: W_2), Grad: None
|  Prev Tensor: Tensor(shape=[1,2]) (Name: B_2), Grad: None


In [13]:
h_2.backward([0.1, 0.3])

In [14]:
logTree(h_2);

Gradient computation graph:
Tensor: Tensor(shape=[1,2]) (Name: h_2)
|  Prev Tensor: Tensor(shape=[1,2]) (Name: undefined), Grad: 0.1,0.3
|  |  Prev Tensor: Tensor(shape=[1,8]) (Name: h_1), Grad: 0.22406068444252014,0.04818158596754074,-0.11016245931386948,-0.21478025615215302,0.5761716961860657,0.42526674270629883,-0.2083713561296463,-0.03610248491168022
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: undefined), Grad: 0.22406068444252014,0.04818158596754074,-0.11016245931386948,-0.21478025615215302,0.5761716961860657,0.42526674270629883,-0.2083713561296463,-0.03610248491168022
|  |  |  |  Prev Tensor: Tensor(shape=[1,4]) (Name: Input (x)), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[4,8]) (Name: W_1), Grad: 1.5684248208999634,0.3372710943222046,-0.7711372375488281,-1.5034618377685547,4.033201694488525,2.976867198944092,-1.4585994482040405,-0.25271740555763245,1.7924854755401611,0.3854526877403259,-0.8812996745109558,-1.7182420492172241,4.609373569488525,3.4021339416503906,-1.66